In [1]:
import csv

import numpy as np

try:
    from gensim import models
except ModuleNotFoundError as e:
    !pip install gensim==3.8.0
    from gensim import models
try:
    import pandas as pd
except ModuleNotFoundError as e:
    !pip install pandas
    import pandas as pd
    
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError as e:
    !pip install matplitlib
    import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# This line prevents TF crashing when using convolutional networks
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
def prepare_data(model_path, data_prefix, seq_len, embedding_size):
    """
    
    Args:
        model_path (str): path to embedding model
        data_prefix (str): path to data prefix
    
    """
    
    def load_data(path):
        output = []
        
        with open(path, 'r', encoding='latin-1') as data:
        #with open(path, 'r', encoding='utf-8') as data:
            for row in csv.reader(data):
                output.append(row)
                
        return output
    
    # Opens embedding model
    model_ = models.Word2Vec.load(model_path)
    
    # Open dataset
    data_train = load_data(data_prefix + "X_train.csv")
    data_val = load_data(data_prefix + "X_val.csv")
    label_train = np.loadtxt(data_prefix + "y_train.csv")
    label_val = np.loadtxt(data_prefix + "y_val.csv")
    
    # Gets embeddings from model
    dt = []
    lt = []
    omissions_ = 0
    
    for i, seq in enumerate(data_train):
        
        try:        
            embedding = model_.wv[seq]
            dt.append(embedding)
            lt.append(label_train[i])
        
        except KeyError as ke:
            for word in seq:
                if word not in model_.wv.vocab.keys():
                    seq.remove(word)
            
        except ValueError as ve:
            omissions_ += 1
            
#     print(omissions_)
    
    # Gets embeddings from model
    dv = []
    lv = []
    omissions_ = 0
    
    for i, seq in enumerate(data_val):
        
        try:
            embedding = model_.wv[seq]
            dv.append(embedding)
            lv.append(label_val[i])
        
        except KeyError as ke:
            for word in seq:
                if word not in model_.wv.vocab.keys():
                    seq.remove(word)
            
        except ValueError as ve:
            omissions_ += 1
    
#     print(omissions_)
    
    # Pads sequences
    dt = pad_sequences(dt, padding='post', dtype='float64', maxlen=seq_len)
    dv = pad_sequences(dv, padding='post', dtype='float64', maxlen=seq_len)
    
    # Converts lists to numpy arrays
#     dt = np.asarray(dt).reshape((len(dt), seq_len * embedding_size))
#     dv = np.asarray(dv).reshape((len(dv), seq_len * embedding_size))
    
    lt = np.asarray(lt)
    lv = np.asarray(lv)
    
    return dt, dv, lt, lv

In [4]:
simpson_dict = {5: "./resources/embeddings/Simpsons_5_7.model",
                25: "./resources/embeddings/Simpsons_25_7.model",
                125: "./resources/embeddings/Simpsons_125_7.model",
                300: "./resources/embeddings/Simpsons_300_7.model",
                'prefix': "./data/simpsons/",
                'classes': 4}

friends_dict = {5: "./resources/embeddings/Friends_5_7.model",
                25: "./resources/embeddings/Friends_25_7.model",
                125: "./resources/embeddings/Friends_125_7.model",
                'prefix': "./data/friends/",
                'classes': 6}

In [5]:
def create_checkpoint_callback(filepath):
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = filepath,
        save_weights_only = False,
        monitor = 'accuracy',
        mode = 'max',
        save_best_only = True)
    return model_checkpoint_callback

In [26]:
def create_early_stop_callback():
    callback = tf.keras.callbacks.EarlyStopping(
    monitor = tf.keras.metrics.SparseCategoricalAccuracy(), min_delta = 0, patience = 15, verbose = 0,
    mode = 'max', baseline = None, restore_best_weights = False)
    return callback

## Model creators

In [7]:
def create_base_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.Dense((embedding_size * seq_len)/2, activation="relu", input_shape = input_shape),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [8]:
def create_deepFC_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.Dense((embedding_size * seq_len)/2, activation="relu", input_shape = input_shape),
        layers.Dropout(0.2),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [9]:
def create_RNN_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.SimpleRNN(units = 75, input_shape=input_shape),
#         layers.SimpleRNN(units = 20, return_sequences=True),
#         layers.SimpleRNN(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [10]:
def create_LSTM_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.LSTM(units = 75, input_shape = input_shape),
#         layers.LSTM(units = 20, return_sequences=True),
#         layers.LSTM(units = 20, return_sequences=True),
        layers.Flatten(),
        layers.Dense((embedding_size * seq_len)/2, activation="relu"),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall()]
    )
    model.summary()
    return model

In [11]:
model_creators = {'baseline': create_base_model,
                  'DeepFC': create_deepFC_model,
                  'SimpleRNN': create_RNN_model,
                  'LSTM': create_LSTM_model}

## Training methods

In [13]:
def train_val_model(dataset, em_size, seq_len, model_type, epochs):
    if dataset == 'simpson':
        data_path = simpson_dict
    elif dataset == 'friends':
        data_path = friends_dict
    else: 
        raise 'Not valid dataset'
        
    X_train, X_val, y_train, y_val = prepare_data(data_path[em_size], data_path['prefix'],
                                                  seq_len, em_size)
    
    if model_type in  ['baseline', 'DeepFC']:
        X_train = np.asarray(X_train).reshape((len(X_train), seq_len * em_size))
        X_val = np.asarray(X_val).reshape((len(X_val), seq_len * em_size))
    
    checkpoint_callback = create_checkpoint_callback('./resources/checkpoints/' + model_type + '_' + 
                                          str(em_size) + '_' +str(seq_len))
    
    early_stop_callback = create_early_stop_callback()
    
    input_shape = (len(X_train), seq_len * em_size) if model_type in ['baseline','DeepFC'] else (seq_len, em_size)
    
    model = model_creators[model_type](em_size, seq_len, data_path['classes'], 
                                       input_shape=input_shape)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), 
              epochs = epochs, verbose = 0, 
              callbacks=[checkpoint_callback, early_stop_callback])
    
    model_metrics_train = model.evaluate(x = X_train, y = y_train)
    model_metrics_val = model.evaluate(x = X_val, y = y_val)
    data = {'model_name': [model_type], 'embedding_size': [em_size], 'seq_len': [seq_len],
            'train_accuracy': [model_metrics_train[1]], 'train_precision': [model_metrics_train[2]], 'train_recall': [model_metrics_train[3]],
           'val_accuracy': [model_metrics_val[1]], 'val_precision': [model_metrics_val[2]], 'val_recall': [model_metrics_val[3]]}
    
    return pd.DataFrame(data=data)

## Model Training

In [13]:
SEQ_LEN_ = [8, 15, 25, 30, 50]
EM_SIZE_ = [5,25,125,300]
MODEL_TYPES_ = ['baseline','DeepFC', 'SimpleRNN', 'LSTM']
metrics_df = pd.DataFrame()
for seq_len in SEQ_LEN_:
    for em_size in EM_SIZE_:
        for model_type in MODEL_TYPES_:
            model_results = train_val_model('simpson', em_size, seq_len, model_type, 200)
            metrics_df = pd.concat([metrics_df, model_results])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 45326, 20)         820       
_________________________________________________________________
dense_1 (Dense)              (None, 45326, 4)          84        
Total params: 904
Trainable params: 904
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
257/257 [==============================] - 0s 

INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


...

INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_300_50/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_300_50/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_300_50/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_300_50/assets


131/131 [==============================] - 0s 3ms/step - loss: 1.3075 - accuracy: 0.4282 - precision_79: 0.5000 - recall_79: 2.3895e-04


In [14]:
metrics_df

,model_name,embedding_size,seq_len,train_accuracy,train_precision,train_recall,val_accuracy,val_precision,val_recall
0,baseline,5,8,0.428430,0.555178,0.039624,0.432261,0.481611,0.033504
0,DeepFC,5,8,0.428231,0.801296,0.008185,0.435916,0.740260,0.006944
0,SimpleRNN,5,8,0.447712,0.615921,0.134514,0.434820,0.507254,0.110746
0,LSTM,5,8,0.486564,0.639733,0.201055,0.444810,0.516215,0.164839
0,baseline,25,8,0.538852,0.685975,0.294709,0.424708,0.498830,0.207724
...,...,...,...,...,...,...,...,...,...
0,LSTM,125,50,0.607113,0.723278,0.456074,0.507797,0.593097,0.366350
0,baseline,300,50,0.675297,0.800209,0.535468,0.428435,0.474155,0.291517
0,DeepFC,300,50,0.676893,0.851505,0.489724,0.426762,0.491351,0.251135
0,SimpleRNN,300,50,0.421680,1.000000,0.000150,0.428196,0.000000,0.000000


In [15]:
metrics_df.to_csv('metrics_df.csv')

In [14]:
SEQ_LEN_ = [8, 15, 25, 30, 50]
EM_SIZE_ = [5, 25, 125]
MODEL_TYPES_ = ['baseline','DeepFC', 'SimpleRNN', 'LSTM']
metrics_df = pd.DataFrame()
for seq_len in SEQ_LEN_:
    for em_size in EM_SIZE_:
        for model_type in MODEL_TYPES_:
            model_results = train_val_model('friends', em_size, seq_len, model_type, 200)
            metrics_df = pd.concat([metrics_df, model_results])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32515, 20)         820       
_________________________________________________________________
dense_1 (Dense)              (None, 32515, 6)          126       
Total params: 946
Trainable params: 946
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_5_8/assets
INFO:tensorflow:Assets written to: ./resources

INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/LSTM_5_8/assets


...

INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_30/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_30/assets


INFO:tensorflow:Assets written to: ./resources/checkpoints/baseline_125_30/assets


 266/1017 [======>.......................] - ETA: 1s - loss: 0.4701 - accuracy: 0.8400 - precision_44: 0.9412 - recall_44: 0.7817

KeyboardInterrupt: 

In [67]:
def create_LSTM_model(embedding_size, seq_len, classes, input_shape):
    model = keras.Sequential([
        layers.LSTM(units = 20, input_shape=input_shape),
        layers.Flatten(),
        layers.Dense(int((embedding_size * seq_len)/10), activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(classes, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, 
                                           beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )
    model.summary()
    return model

In [ ]:
dataset='simpson'
em_size=25
seq_len=10
model_type='LSTM'
epochs=1000

X_train, X_val, y_train, y_val = prepare_data(simpson_dict[em_size], simpson_dict['prefix'],
                                              seq_len, em_size)

checkpoint_callback = create_checkpoint_callback('./resources/checkpoints/' + model_type + '_' + 
                                      str(em_size) + '_' +str(seq_len))
early_stop_callback = create_early_stop_callback()

input_shape = (seq_len, em_size)

model = create_LSTM_model(em_size, seq_len, simpson_dict['classes'], input_shape=input_shape)

history = model.fit(X_train, np.argmax(y_train, axis=1).reshape(-1,1),
                    validation_data=(X_val, np.argmax(y_val, axis=1).reshape(-1,1)), 
                    epochs = epochs, verbose = 1)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 20)                3680      
_________________________________________________________________
flatten_14 (Flatten)         (None, 20)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 25)                525       
_________________________________________________________________
dropout_2 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 4)                 104       
Total params: 4,309
Trainable params: 4,309
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
1417/1417 [==============================] - 6s 3ms/step - loss: 1.3484 - sparse_categorical_accuracy: 0

Epoch 40/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.2005 - sparse_categorical_accuracy: 0.4767 - val_loss: 1.2088 - val_sparse_categorical_accuracy: 0.4773
Epoch 41/1000
1417/1417 [==============================] - 6s 4ms/step - loss: 1.1995 - sparse_categorical_accuracy: 0.4749 - val_loss: 1.2072 - val_sparse_categorical_accuracy: 0.4779
Epoch 42/1000
1417/1417 [==============================] - 5s 4ms/step - loss: 1.2000 - sparse_categorical_accuracy: 0.4757 - val_loss: 1.2070 - val_sparse_categorical_accuracy: 0.4786
Epoch 43/1000
1417/1417 [==============================] - 5s 3ms/step - loss: 1.2003 - sparse_categorical_accuracy: 0.4756 - val_loss: 1.2058 - val_sparse_categorical_accuracy: 0.4790
Epoch 44/1000
1417/1417 [==============================] - 6s 4ms/step - loss: 1.1986 - sparse_categorical_accuracy: 0.4734 - val_loss: 1.2061 - val_sparse_categorical_accuracy: 0.4805
Epoch 45/1000
1417/1417 [==============================] - 5s 4ms/step - lo

1417/1417 [==============================] - 7s 5ms/step - loss: 1.1753 - sparse_categorical_accuracy: 0.4899 - val_loss: 1.2010 - val_sparse_categorical_accuracy: 0.4821
Epoch 85/1000
1417/1417 [==============================] - 7s 5ms/step - loss: 1.1731 - sparse_categorical_accuracy: 0.4861 - val_loss: 1.1998 - val_sparse_categorical_accuracy: 0.4826
Epoch 86/1000
1417/1417 [==============================] - 6s 4ms/step - loss: 1.1725 - sparse_categorical_accuracy: 0.4873 - val_loss: 1.1990 - val_sparse_categorical_accuracy: 0.4850
Epoch 87/1000
1417/1417 [==============================] - 7s 5ms/step - loss: 1.1683 - sparse_categorical_accuracy: 0.4878 - val_loss: 1.1994 - val_sparse_categorical_accuracy: 0.4837
Epoch 88/1000
1417/1417 [==============================] - 7s 5ms/step - loss: 1.1665 - sparse_categorical_accuracy: 0.4921 - val_loss: 1.1993 - val_sparse_categorical_accuracy: 0.4815
Epoch 89/1000
1417/1417 [==============================] - 5s 4ms/step - loss: 1.1672 - s

1417/1417 [==============================] - 7s 5ms/step - loss: 1.1607 - sparse_categorical_accuracy: 0.4933 - val_loss: 1.2000 - val_sparse_categorical_accuracy: 0.4837
Epoch 129/1000
1417/1417 [==============================] - 5s 4ms/step - loss: 1.1559 - sparse_categorical_accuracy: 0.4962 - val_loss: 1.1993 - val_sparse_categorical_accuracy: 0.4867
Epoch 130/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.1573 - sparse_categorical_accuracy: 0.4957 - val_loss: 1.2022 - val_sparse_categorical_accuracy: 0.4814
Epoch 131/1000
1417/1417 [==============================] - 5s 3ms/step - loss: 1.1620 - sparse_categorical_accuracy: 0.4937 - val_loss: 1.2011 - val_sparse_categorical_accuracy: 0.4851
Epoch 132/1000
1417/1417 [==============================] - 5s 4ms/step - loss: 1.1602 - sparse_categorical_accuracy: 0.4937 - val_loss: 1.1998 - val_sparse_categorical_accuracy: 0.4864
Epoch 133/1000
1417/1417 [==============================] - 5s 4ms/step - loss: 1.161

1417/1417 [==============================] - 4s 3ms/step - loss: 1.1485 - sparse_categorical_accuracy: 0.5045 - val_loss: 1.2067 - val_sparse_categorical_accuracy: 0.4825
Epoch 173/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.1547 - sparse_categorical_accuracy: 0.4996 - val_loss: 1.2043 - val_sparse_categorical_accuracy: 0.4831
Epoch 174/1000
1417/1417 [==============================] - 5s 3ms/step - loss: 1.1456 - sparse_categorical_accuracy: 0.5016 - val_loss: 1.2030 - val_sparse_categorical_accuracy: 0.4843
Epoch 175/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.1502 - sparse_categorical_accuracy: 0.5029 - val_loss: 1.2065 - val_sparse_categorical_accuracy: 0.4838
Epoch 176/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.1473 - sparse_categorical_accuracy: 0.5019 - val_loss: 1.2043 - val_sparse_categorical_accuracy: 0.4842
Epoch 177/1000
1417/1417 [==============================] - 4s 3ms/step - loss: 1.143

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.legend(['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy'])

In [38]:
history.history.keys()

dict_keys(['loss', 'sparse_categorical_accuracy', 'val_loss', 'val_sparse_categorical_accuracy'])